In [1]:
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from munch import Munch
from tqdm import tqdm_notebook as tqdm

import datasets
import models
import transforms
import utils

In [2]:
config_path = 'pretrained/drn_d_22_OilChange/config.yml'

In [3]:
with open(config_path, 'r') as f:
    cfg = Munch.fromYAML(f)

In [4]:
model = models.DRNSeg(cfg.arch, cfg.data.classes, None, pretrained=True)
model = torch.nn.DataParallel(model).cuda()
cudnn.benchmark = True
checkpoint = torch.load(cfg.training.resume)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print("=> loaded checkpoint '{}' (epoch {})".format(cfg.training.resume, checkpoint['epoch']))

=> loaded checkpoint 'pretrained/drn_d_22_OilChange/checkpoint_00000900.pth.tar' (epoch 900)


In [5]:
val_transforms = transforms.Compose([transforms.ToTensor()])

In [6]:
val_dataset = datasets.Dataset(cfg.data.root, 'val_' + cfg.data.ann_file, 'val', val_transforms)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [7]:
acc = utils.AverageMeter()
hist = np.zeros((cfg.data.classes, cfg.data.classes))
with torch.no_grad():
    for input, target in tqdm(val_loader):
        target = target.cuda(non_blocking=True)
        output = model(input)
        acc.update(utils.accuracy(output, target), input.size(0))
        _, pred = output.max(1)
        hist += utils.fast_hist(pred.cpu().data.numpy().flatten(), target.cpu().numpy().flatten(), cfg.data.classes)
ious = utils.per_class_iou(hist)

print('accuracy: {:.2f}'.format(acc.val))
print('mean IoU: {:.4f}'.format(ious.mean()))


accuracy: 99.82
mean IoU: 0.9650
